# Задача классификации текста

В этом модуле мы начнем с простой задачи классификации текста на основе набора данных **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: будем классифицировать заголовки новостей в одну из 4 категорий: Мир, Спорт, Бизнес и Наука/Технологии.

## Набор данных

Для загрузки набора данных мы будем использовать API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Теперь мы можем получить доступ к обучающей и тестовой частям набора данных, используя `dataset['train']` и `dataset['test']` соответственно:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Давайте выведем первые 10 новых заголовков из нашего набора данных:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Векторизация текста

Теперь нам нужно преобразовать текст в **числа**, которые можно представить в виде тензоров. Если мы хотим представление на уровне слов, нужно выполнить два шага:

* Использовать **токенизатор**, чтобы разбить текст на **токены**.
* Построить **словарь** из этих токенов.

### Ограничение размера словаря

В примере с набором данных AG News размер словаря довольно большой — более 100 тысяч слов. В общем случае, нам не нужны слова, которые редко встречаются в тексте — они будут присутствовать только в нескольких предложениях, и модель не сможет на них обучиться. Поэтому имеет смысл ограничить размер словаря до меньшего числа, передав соответствующий аргумент в конструктор векторизатора:

Оба этих шага можно выполнить с помощью слоя **TextVectorization**. Давайте создадим объект векторизатора, а затем вызовем метод `adapt`, чтобы пройтись по всему тексту и построить словарь:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Примечание**: мы используем только часть всего набора данных для построения словаря. Это делается для ускорения выполнения и чтобы не заставлять вас ждать. Однако существует риск, что некоторые слова из полного набора данных не попадут в словарь и будут проигнорированы во время обучения. Таким образом, использование полного размера словаря и обработка всего набора данных во время `adapt` может повысить итоговую точность, но незначительно.

Теперь мы можем получить доступ к фактическому словарю:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Используя векторизатор, мы можем легко преобразовать любой текст в набор чисел:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Представление текста методом "мешка слов"

Поскольку слова несут смысл, иногда можно понять значение текста, просто анализируя отдельные слова, независимо от их порядка в предложении. Например, при классификации новостей слова, такие как *погода* и *снег*, скорее всего, укажут на *прогноз погоды*, тогда как слова, такие как *акции* и *доллар*, будут относиться к *финансовым новостям*.

**Мешок слов** (BoW) — это самое простое для понимания традиционное представление в виде вектора. Каждое слово связано с индексом вектора, а элемент вектора содержит количество вхождений каждого слова в данном документе.

![Изображение, показывающее, как представление вектора методом "мешка слов" хранится в памяти.](../../../../../translated_images/ru/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Метод BoW также можно рассматривать как сумму всех векторов с одним активным элементом (one-hot-encoded), соответствующих отдельным словам в тексте.

Ниже приведен пример того, как создать представление методом "мешка слов" с использованием библиотеки Scikit Learn на Python:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Мы также можем использовать векторизатор Keras, который мы определили выше, преобразуя каждый номер слова в one-hot кодирование и складывая все эти векторы.


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Примечание**: Вас может удивить, что результат отличается от предыдущего примера. Причина в том, что в примере с Keras длина вектора соответствует размеру словаря, который был создан на основе всего набора данных AG News, тогда как в примере с Scikit Learn мы создавали словарь на лету, используя текст выборки.


## Обучение классификатора BoW

Теперь, когда мы научились создавать представление текста в виде мешка слов, давайте обучим классификатор, который будет его использовать. Сначала нам нужно преобразовать наш набор данных в представление мешка слов. Это можно сделать с помощью функции `map` следующим образом:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Теперь давайте определим простой классификатор нейронной сети, который содержит один линейный слой. Размер входа — `vocab_size`, а размер выхода соответствует количеству классов (4). Поскольку мы решаем задачу классификации, конечная функция активации — **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Поскольку у нас есть 4 класса, точность выше 80% считается хорошим результатом.

## Обучение классификатора как одной сети

Поскольку векторизатор также является слоем Keras, мы можем определить сеть, которая включает его, и обучить её от начала до конца. Таким образом, нам не нужно векторизовать набор данных с помощью `map`, мы можем просто передать оригинальный набор данных на вход сети.

> **Примечание**: Нам всё же потребуется применять `map` к нашему набору данных, чтобы преобразовать поля из словарей (таких как `title`, `description` и `label`) в кортежи. Однако, при загрузке данных с диска, мы можем сразу создать набор данных с необходимой структурой.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Биграммы, триграммы и n-граммы

Одно из ограничений подхода "мешок слов" заключается в том, что некоторые слова являются частью многословных выражений. Например, слово "hot dog" имеет совершенно другое значение, чем слова "hot" и "dog" в других контекстах. Если мы всегда представляем слова "hot" и "dog" с помощью одинаковых векторов, это может запутать нашу модель.

Чтобы решить эту проблему, часто используются **представления n-грамм** в методах классификации документов, где частота каждого слова, двухсловных или трехсловных выражений является полезной характеристикой для обучения классификаторов. В представлениях биграмм, например, мы добавляем все пары слов в словарь, помимо оригинальных слов.

Ниже приведен пример того, как создать представление "мешок слов" для биграмм с использованием Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основной недостаток подхода n-грамм заключается в том, что размер словаря начинает расти чрезвычайно быстро. На практике нам необходимо сочетать представление n-грамм с техникой уменьшения размерности, такой как *встраивания* (embeddings), о которых мы поговорим в следующем разделе.

Чтобы использовать представление n-грамм в нашем наборе данных **AG News**, нам нужно передать параметр `ngrams` в конструктор `TextVectorization`. Длина словаря биграмм **значительно больше**, в нашем случае это более 1,3 миллиона токенов! Поэтому имеет смысл ограничить количество токенов биграмм до какого-то разумного числа.

Мы могли бы использовать тот же код, что и выше, для обучения классификатора, однако это было бы крайне неэффективно с точки зрения памяти. В следующем разделе мы будем обучать классификатор биграмм с использованием встраиваний. А пока вы можете поэкспериментировать с обучением классификатора биграмм в этой тетради и попробовать добиться более высокой точности.


## Автоматический расчет векторов BoW

В приведенном выше примере мы вручную рассчитывали векторы BoW, суммируя one-hot представления отдельных слов. Однако последняя версия TensorFlow позволяет автоматически рассчитывать векторы BoW, передавая параметр `output_mode='count` в конструктор векторизатора. Это значительно упрощает определение и обучение нашей модели:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Частота термина - обратная частота документа (TF-IDF)

В представлении BoW частота слов взвешивается одинаково, независимо от самого слова. Однако очевидно, что часто встречающиеся слова, такие как *a* и *in*, гораздо менее важны для классификации, чем специализированные термины. В большинстве задач обработки естественного языка некоторые слова более значимы, чем другие.

**TF-IDF** расшифровывается как **частота термина - обратная частота документа**. Это вариация метода "мешок слов", где вместо бинарного значения 0/1, указывающего на наличие слова в документе, используется значение с плавающей точкой, связанное с частотой появления слова в корпусе.

Более формально, вес $w_{ij}$ слова $i$ в документе $j$ определяется как:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
где
* $tf_{ij}$ — количество появлений $i$ в $j$, то есть значение BoW, которое мы рассматривали ранее
* $N$ — количество документов в коллекции
* $df_i$ — количество документов, содержащих слово $i$ во всей коллекции

Значение TF-IDF $w_{ij}$ увеличивается пропорционально количеству раз, которое слово появляется в документе, и корректируется с учетом количества документов в корпусе, содержащих это слово. Это помогает учитывать тот факт, что некоторые слова встречаются чаще других. Например, если слово появляется *в каждом* документе коллекции, $df_i=N$, и $w_{ij}=0$, такие термины полностью игнорируются.

Вы можете легко создать векторизацию текста с использованием TF-IDF через Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

В Keras слой `TextVectorization` может автоматически вычислять частоты TF-IDF, передавая параметр `output_mode='tf-idf'`. Давайте повторим код, который мы использовали выше, чтобы проверить, увеличивает ли использование TF-IDF точность:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Заключение

Хотя представления TF-IDF предоставляют весовые коэффициенты для различных слов, они не способны передавать значение или порядок. Как сказал известный лингвист Дж. Р. Фёрс в 1935 году: "Полное значение слова всегда контекстуально, и никакое изучение значения вне контекста не может быть воспринято всерьез." Позже в этом курсе мы узнаем, как извлекать контекстуальную информацию из текста с помощью языкового моделирования.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
